# Hypothesis Testing in Python

Hypothesis tests check if the sample statistics lie in the tails of the **null distribution**
- two-tailed: alternative different from null
- right-tailed: alternative greater than null 
- left-tailed: alternative less than null
H(A): The proportion of data scientists starting programming as children is **greater than** 35%
This is a **right-tailed** test(A)

## p-values
**p-values**: probability of obtaining a result, assuming the null hypothesis is true
1. Large p-value, large support for H(O)
- Statistic likely **not in** the tail of the nulldistribution
2. Small p-value, strong evidence against H(O)
- Statistic likely **in** the tail of the _null distribution_ 
3. "p" in p-value → probability
4. "small" means "close to zero"00

### Calculating the p-value
- **norm.cdf()** is normal CDF from **scipy.stats**
- Left-tailed test --> use **norm.cdf()**
- Right-tailed test --> **1 - norm.cdf()**

In [ ]:
# p-value
from scipy.stats import norm
p_value = 1 - norm.cdf(z_score, loc=0, scale=1) 

## Significance level
The significance level of a hypothesis test (α) is the threshold point for "beyond a reasonable doubt"
- Common values of α are 0.2, 0.1, 0.05, and 0.01
- If p≤α, reject H, else fail to reject H
- α should be set prior to conducting the hypothesis test00


In [ ]:
# Making a decision
alpha = 0.05
print(p_value <= alpha)

## Confidence intervals
For a significance level of α, it's common to choose a confidence interval level of 1 - α
- α=0.05 → 95% confidence interval

In [ ]:
# Confidence intervals
import numpy as np
lower = np.quantile(first_code_boot_distn, 0.025)
upper = np.quantile(first_code_boot_distn, 0.975)
print((lower, upper))

## Performing t-tests

In [ ]:
# Calculations assuming the null hypothesis is true
xbar = stack_overflow.groupby('age_first_code_cut')['converted_comp'].mean()
s = stack_overflow.groupby('age_first_code_cut')['converted_comp'].std()
n = stack_overflow.groupby('age_first_code_cut')['converted_comp'].count()

# Calculating the test statistic
import numpy as np
numerator = xbar_child - xbar_adult
denominator = np.sqrt(s_child ** 2 / n_child + s_adult ** 2 / n_adult)
t_stat = numerator / denominator

# Calculating the degrees of freedom
degrees_of_freedom = n_child + n_adult - 2

# Calculating p-values from t-values
from scipy.stats import t
1 - t.cdf(t_stat, df=degrees_of_freedom)

## Paired t-tests

In [ ]:
# Testing differences between two means using ttest()
import pingouin
pingouin.ttest(x=sample_data['diff'],
               y=0,
               alternative="less")

# ttest() with paired=True
pingouin.ttest(x=sample_data['repub_percent_08'],               
               y=sample_data['repub_percent_12'],
               paired=True,
               alternative="less")

# Unpaired ttest()
pingouin.ttest(x=sample_data['repub_percent_08'],               
               y=sample_data['repub_percent_12'],
               paired=False, # The default
               alternative="less")

## ANOVA tests

### Methods without example
**Method used for testing and adjustment of pvalues.**
- **'none'**: no correction (default)
- **'bonf'**: one-step Bonferroni correction
- **'sidak'**: one-step Sidak correction
- **'holm'**: step-down method using Bonferroni adjustments
- **'fdr_bh'**: Benjamini/Hochberg FDR correction
- **'fdr_by'**: Benjamini/Yekutieli FDR correction


In [ ]:
# Analysis of variance (ANOVA)

# A test for differences between groups
alpha = 0.2
pingouin.anova(data=stack_overflow,
                          dv="converted_comp",
                          between="job_sat")
# 0.001315 < α
# At least two categories have significantly different compensation

# pairwise_tests()
pingouin.pairwise_tests(data=stack_overflow,                         
                        dv="converted_comp",                         
                        between="job_sat",
                        padjust="none")
# Bonferroni correction
pingouin.pairwise_tests(data=stack_overflow,                         
                        dv="converted_comp",
                        between="job_sat",
                        padjust="bonf")

## Proportion tests

### One-sample proportion tests

In [ ]:
# Caculating the z-score
import numpy as np
numerator = p_hat - p_0
denominator = np.sqrt(p_0 * (1 - p_0) / n)
z_score = numerator / denominator

# Calculating the p_value
from scipy.stats import norm

# Left-tailed ("less than")
p_value = norm.cdf(z_score)

# Right-tailed ("greater than")
p_value = 1 - norm.cdf(z_score)

# Two-tailed ("not equal")
p_value = norm.cdf(-z_score) + 1 - norm.cdf(z_score)

# If pdf was symmetric
p_value = 2 * (1 - norm.cdf(z_score))

### Two-sample proportion tests

In [ ]:
# Getting the numbers for the z-score
p_hat = (n_at_least_30 * p_hat_at_least_30 + n_under_30 * p_hat_under_30) /         (n_at_least_30 + n_under_30)

std_error = np.sqrt(p_hat * (1-p_hat) / n_at_least_30) + (p_hat * 1-p_hat) / n_under_30))

z_score = (p_hat_at_least_30 - p_hat_under_30) / std_error

print(z_score)

# Proportion tests using proportions_ztest()
n_hobbyists = np.array([812, 1021])
n_rows = np.array([812 + 238, 1021 + 190])
from statsmodels.stats.proportion import proportions_ztest
z_score, p_value = proportions_ztest(count=n_hobbyists, nobs=n_rows,                                     alternative="two-sided")

### Chi-square test of independence

In [ ]:
# Test for independence of variables 
import pingouin
expected, observed, stats = pingouin.chi2_independence(data=stack_overflow, x='hobbyist', y='age_cat', correction=False)

print(stats)

### Chi-square gooodness of fit tests
**Purple links**
- How do you feel when you discover that you've already visited the top resource?

In [ ]:
# Purple links
purple_link_counts = stack_overflow['purple_link'].value_counts()

purple_link_counts = purple_link_counts.rename_axis('purple_link')\                                                      .reset_index(name='n')\                                                            .sort_values('purple_link')

# Declaring the hypotheses
hypothesized = pd.DataFrame({'purple_link': ['Amused', 'Annoyed', 'Hello, old friend', 'Indifferent'], 'prop': [1/6, 1/6, 1/2, 1/6]})

# Hypothesized counts by category
n_total = len(stack_overflow)
hypothesized["n"] = hypothesized["prop"] * n_total

# Visualizing counts
import matplotlib.pyplot as plt

plt.bar(purple_link_counts['purple_link'], purple_link_counts['n'],                       color='red', label='Observed')

plt.bar(hypothesized['purple_link'], hypothesized['n'], alpha=0.5,                         color='blue', label='Hypothesized')

plt.legend()
plt.show()

# chi-square goodness of fit test
from scipy.stats import chisquare

chisquare(f_obs=purple_link_counts['n'], f_exp=hypothesized['n'])

## Non-Parametric Tests

In [ ]:
# Results with pingouin.ttest()
alpha = 0.01
import pingouin
pingouin.ttest(x=repub_votes_potus_08_12_small['repub_percent_08'],                              y=repub_votes_potus_08_12_small['repub_percent_12'],
               paired=True,
               alternative="less")

# Non-parametric tests
x = [1, 15, 3, 10, 6]
from scipy.stats import rankdata
rankdata(x)

# Wilcoxon-signed rank test
# Step 1
repub_votes_small['diff'] = repub_votes_small['repub_percent_08'] - repub_votes_small['repub_percent_12']

# Step 2
repub_votes_small['abs_diff'] = repub_votes_small['diff'].abs()

# Step 3 
from scipy.stats import rankdata
repub_votes_small['rank_abs_diff'] = rankdata(repub_votes_small['abs_diff'])

# Step 4
T_minus = 1 + 4 + 5 + 2 + 3
T_plus = 0
W = np.min([T_minus, T_plus])

# Implementation with pingouin.wilcoxon()
alpha = 0.01
pingouin.wilcoxon(x=repub_votes_potus_08_12_small['repub_percent_08'],                               y=repub_votes_potus_08_12_small['repub_percent_12'],
                  alternative="less")

### Non-parametric ANOVA and unpaired t-tests

In [ ]:
# Wilcoxon-Mann_whitney test setup
age_vs_comp = stack_overflow[['converted_comp', 'age_first_code_cut']]
age_vs_comp_wide = age_vs_comp.pivot(columns='age_first_code_cut',
                                     values='converted_comp')

# Wilcoxon-Mann_whitney test
alpha=0.01
import pingouin
pingouin.mwu(x=age_vs_comp_wide['child'],              
             y=age_vs_comp_wide['adult'],
             alternative='greater')

# Kruskal-Wallis test
alpha=0.01
pingouin.kruskal(data=stack_overflow,
                 dv='converted_comp',
                 between='job_sat')

# Preprocessing for Machine Learning in Python

In [ ]:
# Removing missing data
'''
df.drop([], axis=0 or 1)
df.dropna(subset=[])
# specifies the columns that will have their rows deleted 
'''

# Checking column type
'''
df.info() or df.dtypes
'''

# Converting column type
'''
df['column_1'] = df[column_1].astype('float')
'''

In [ ]:
# Solving imbalanced classes with strafied sampling
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y,
                                                    test_size=0.2,
                                                    random_state=42)

### Log normalization
- Useful for features with high variance
- Applies lofarithm transformation
- Natural log using the constant *e*
- Capture relative changes, the magnitude of change, and keeps everything positive

In [ ]:
# Log nomalization in Python
import pandas as pd
df = pd.DataFrame({
    'col1':[1.00, 1.20, 0.75, 1.60],
    'col2':[3.0, 45.5, 28.0, 100.0]
})

print(df.var())

In [ ]:
import numpy as np
df['log_2'] = np.log(df['col2'])
print(df)
print(df[['col1', 'log_2']].var())

### Feature Scaling
- Features on different scales
- Model with linear characteristics
- Center features around 0 and transform to variance of 1
- Transforms to approximately normal distribution

In [ ]:
# Scale data
df = pd.DataFrame({
    'col1':[1.00, 1.20, 0.75, 1.60],
    'col2':[48.0, 45.5, 46.2, 50.0],
    'col3':[100.0, 101.3, 103.5, 104.0]
})

print(df.var())

#### Data leakage: non-training data is used to train the model

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df),
                         columns=df.columns)
print(df_scaled)
print(df_scaled.var())

### Feature Engineering
#### Feature engeneering: Creation of new features from existing ones
- Improve performance
- Insight relationships between features
- Need to understand the data first
- Highly dataset-dependent

In [ ]:
# Encoding categorical variables
users = pd.DataFrame({
    'subscribed':['y', 'n', 'n', 'y']
    'fav_color':['blue', 'green', 'orange', 'green']
})

users['sun_enc'] = users['subscribed'].apply(lambda val: 1 if val=='y' else 0)


# Encoding binary variables with scikit-learn

# Label encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

users['sub_enc_le'] = le.fit_transform(users['subscribed'])
print(users['subscribed', 'sub_enc_le'])

# One-hot encoding
print(pd.get_dummies(users['fav_color']))

### Engineering numerical features

In [ ]:
# Descriptive Statistics
temps = pd.DataFrame({
    'city':['NYC', 'SF', 'LA', 'Boston'],
    'day1':[68.3, 75.1, 80.3, 63.0],
    'day2':[67.9, 75.5, 84.0, 61.0],
    'day3':[67.8, 74.9, 81.3, 61.2]
})

temps['mean'] = temps.loc[:, 'day1':'day3'].mean(axis=1)
print(temps)

In [ ]:
# Dates
purchases = pd.DataFrame({
    'date':['July 30 2011', 'February 01 2011', 'January 29 2011', 'March 31 2012', 'February 5 2011'],
    'purchase':['$45.08', '$19.48', '$76.09', '$32.61', '$75.98']
})

purchases['date_converted'] = pd.to_datetime(purchases['date'])
purchases['month'] = purchases['date_converted'].dt.month
print(purchases)

### Engineering text features
#### Regular expressions: code to identify patterns

In [ ]:
# Extraction
import re
my_string = 'temperature:75.6 F'
temp = re.search('\d+\.\d+', my_string)

print(float(temp.group(0)))

### Vectorizing text
#### TF/IDF: Vectorizes words based upon importance
- TF = Term Frequency
- IDF = Inverse Document Frequency

In [ ]:
# Vectorizing text
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
text_tfidf = tfidf_vec.fit_transform(documents)

### Feature Selection
- Reduce noise
- Features with strongly statistically correlated
- Reduce overall variance

### Redudant Features
- Remove noisy features
- Remove correlated features
- Remove duplicated features

### Correlated Features
- Statiscally correlated: features move together directionally
- Linear models assume feature independence
- Pearson's correlation coefficient

In [ ]:
# Correlated features
df = pd.DataFrame({
    'A':[3.06, 2.76, 3.24],
    'B':[3.92, 3.40, 3.17],
    'C':[1.04, 1.05, 1.03]
})

print(df.corr())

 ### Selecting features using text vectors

In [ ]:
# Looking at word weights
print(tfidf_vec.vocabulary_)

print(text_tfidf[3].data)

print(text_tfidf[3].indices)

In [ ]:
vocab = {v:k for k,v in tfidf_vec.vocabulary_.items()}
print(vocab)

In [ ]:
def return_weights(vocab, vector, vector_index):
    zipped = dict(zip(vector[vector_index].indices,
                      vector[vector_index].data))
    
    return {vocab[i]:zipped[i] for i in vector[vector_index].indices}

print(return_weights(vocab, text_tfidf, 3))

### Dimensionality reduction
- Unsupervised learning method
- Combines/decomposes a feature space
- Feature extraction - here we'll use to reduce our feature space

### PCA
- Principal component analysis
- Linear transformation to uncorrelated space
- Captures as much variance as possible in each component

#### PCA caveats
- Difficult to interpret components
- End of preprocessing journey

In [ ]:
# PCA in scikit-learn
from sklearn.decomposition import PCA
pca = PCA()
df_pca = pca.fit_transform(df)

print(df_pca)
print(pca.explained_variance_ratio_)

# Supervised Learning with Scikit-learn

### Naming conventions
- Feature = predictor variable = independent variable
- Target variable = response variable = dependent variable 

In [ ]:
# scikit-learn syntax
from sklearn.module import Model

model = Model()
model.fit(X, y)

predictions = model.predict(X_new)

print(predictions)

### Classifying labels of unseen data
1. Build a model
2. Model learns from the labeled data we pass to it 
3. Pass unlabeled data to the model as input 
4. Model predicts the labels of the unseen data 
- Labeled data = training data

In [ ]:
# k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
X = churn_df[["total_day_charge", "total_eve_charge"]].values
y = churn_df["churn"].values

print(X.shape, y.shape)

# Using scikit-learn to fit a classifier
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X, y)

In [ ]:
# Prediciting on unlabeled data
X_new = np.array([[56.8, 17.5],
                  [24.4, 24.1],
                  [50.1, 10.9]])
print(X_new.shape)

predictions = knn.predict(X_new)
print('Predictions: {}'.format(predictions))

### Measuring Model Performance
- Larger k = less complex model = can cause underfitting
- Smaller k = more complex model = can lead to overfitting

In [ ]:
# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=21,
                                                    stratify=y)
knn = KNeighborsClassifier(n_neighbors=6)

knn.fit(X_train, y_train)

print(knn.score(X_test, y_test))

In [ ]:
# Model complexity and over/underfitting
train_accuracies = {}
test_accuracies = {}
neighbors = np.arange(1, 26)
for neighbor in neighbors:
    knn = KNeighborsClassifier(n_neighbors=neighbor)
    knn.fit(X_train, y_train)
    train_accuracies[neighbor] = knn.score(X_train, y_train)
    test_accuracies[neighbor] = knn.score(X_test, y_test)

In [ ]:
# Plotting our results
plt.figure(figsize=(8, 6))
plt.title("KNN: Varying Number of Neighbors")
plt.plot(neighbors, train_accuracies.values(), label="Training Accuracy")
plt.plot(neighbors, test_accuracies.values(), label="Testing Accuracy")
plt.legend()
plt.xlabel("Number of Neighbors")
plt.ylabel("Accuracy")
plt.show()


## Introduction to Regression

In [ ]:
# Fitting a regression model
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_bmi, y)
predictions = reg.predict(X_bmi)

# Plotting the results
plt.scatter(X_bmi, y)
plt.plot(X_bmi, predictions)
plt.ylabel('Blood Glucose (mg/dl)')
plt.xlabel('Body Mass Index')
plt.show()

## R-squared, MSE and RMSE

In [ ]:
# Linear Regression using all faetures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)
y_pred = reg_all.predict(X_test)

# R-squared in scikit-learn
print(reg_all.score(X_test, y_test))

# RMSE in scikit-learn
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred, squared=False))

## Cross-validation

In [ ]:
# Cross-validation in scikit-learn
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=6, shuffle=True, random_state=42)
reg = LinearRegression()
cv_results = cross_val_score(reg, X, y, cv=kf)

# Evaluating cross-validation performance
print(cv_results)
print(np.mean(cv_results), np.std(cv_results))
print(np.quantile(cv_results, [0.025, 0.975]))

## Regularized regression

In [ ]:
# Ridge regression in scikit-learn
from sklearn.linear_model import Ridge
scores = []
for alpha in [0.1, 1.0, 10.0, 100.0, 1000.0]:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    scores.append(ridge.score(X_test, y_test))
print(scores)

In [ ]:
# Lasso for feature selection in scikit-learn
from sklearn.linear_model import Lasso
X = diabetes_df.drop("glucose", axis=1).values
y = diabetes_df["glucose"].values

names = diabetes_df.drop("glucose", axis=1).columns
lasso = Lasso(alpha=0.1)
lasso_coef = lasso.fit(X, y).coef_

# Plot the results
plt.bar(names, lasso_coef)
plt.xticks(rotation=45)
plt.show()

## Confusion matrix in scikit-learn

In [ ]:
# Confusion matrix and Classification report
from sklearn.metrics import classification_report, confusion_matrix
knn = KNeighborsClassifier(n_neighbors=7)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                    random_state=42)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Logistic regression for binary classification
1. Logistic regression is used for classification problems
2. Logistic regression outputs probabilities
3. If the probability, p > 0.5:
- The datais labeled 1
4. If the probability, p < 0.5:
- The data is labeled 0

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

# Predicting probabilities
y_pred_probs = logreg.predict_proba(X_test)[:, 1]
print(y_pred_probs[0])

## Probability thresholds
1. By default, logistic regression threshold = 0.5
2. Not specific to logistic regression
- KNN classifiers also have thresholds

In [ ]:
# Plotting the ROC curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()

In [ ]:
# ROC AUC in scikit-learn
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred_probs))

## Hyperparameter tuning
- Ridge/lasso regression: Choosing **alpha**
- KNN: Choosing **n_neighbors**
- Hyperparameters: Parameters we specify before fitting the model

### Choosing the correct hyperparameters
1. Try lots of different hyperparameter values
2. Fit all of them separately
3. See how well they perform
4. Choose the best performing values

### Observations:
- This is called **hyperparameter tuning**
- It is essential to use cross-validation to avoid overfitting to the test set
- We can still split the data and perform cross-validation on the training set
- We withhold the test set for final evaluation

In [ ]:
# GridSearhCV
from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {"alpha": np.arange(0.0001, 1, 10),
              "solver": ["sag", "lsqr"]}
ridge = Ridge()
ridge_cv = GridSearchCV(ridge, param_grid, cv=kf)
ridge_cv.fit(X_train, y_train)
print(ridge_cv.best_params_, ridge_cv.best_score_)

In [ ]:
# RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {'alpha': np.arange(0.0001, 1, 10),
              "solver": ['sag', 'lsqr']}
ridge = Ridge()
ridge_cv = RandomizedSearchCV(ridge, param_grid, cv=kf, n_iter=2)
ridge_cv.fit(X_train, y_train)
print(ridge_cv.best_params_, ridge_cv.best_score_)

## Preprocessing data

In [ ]:
# Encoding dummy variables
import pandas as pd
music_df = pd.read_csv('music.csv')

music_dummies = pd.get_dummies(music_df["genre"], drop_first=True)

print(music_dummies.head())

music_dummies = pd.concat([music_df, music_dummies], axis=1)music_dummies = music_dummies.drop("genre", axis=1)

# If the DataFrame only has one categorical feature, we can pass the entire DataFrame:
# music_dummies = pd.get_dummies(music_df, drop_first=True)
# print(music_dummies.columns)

In [ ]:
# Linear regression with dummy variables
from sklearn.model_selection import cross_val_score, KFoldfrom sklearn.linear_model import LinearRegression
X = music_dummies.drop("popularity", axis=1).values
y = music_dummies["popularity"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)
kf = KFold(n_splits=5, 
           shuffle=True, 
           random_state=42)

linreg = LinearRegression()

linreg_cv = cross_val_score(linreg, X_train, y_train, cv=kf,
                            scoring="neg_mean_squared_error")
print(np.sqrt(-linreg_cv))

## Handling missing data

In [ ]:
# Imputation with scikit-learn
from sklearn.impute import SimpleImputer
X_cat = music_df["genre"].values.reshape(-1, 1)
X_num = music_df.drop(["genre", "popularity"], axis=1).values
y = music_df["popularity"].values

X_train_cat, X_test_cat, y_train, y_test = train_test_split(X_cat, y,
test_size=0.2, random_state=12)

X_train_num, X_test_num, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=12)

imp_cat = SimpleImputer(strategy="most_frequent")

X_train_cat = imp_cat.fit_transform(X_train_cat)
X_test_cat = imp_cat.transform(X_test_cat)

In [ ]:
# Imputing within a pipeline
from sklearn.pipeline import Pipeline
music_df = music_df.dropna(subset=["genre", "popularity", "loudness", "liveness", 
                                   "tempo"])

music_df["genre"] = np.where(music_df["genre"] == "Rock", 1, 0)

X = music_df.drop("genre", axis=1).values
y = music_df["genre"].values

steps = [("imputation", SimpleImputer()),
         ("logistic_regression", LogisticRegression())]

pipeline = Pipeline(steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=42)
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

## Centering and scaling

In [ ]:
# Scaling in scikit-learn
from sklearn.preprocessing import StandardScaler
X = music_df.drop("genre", axis=1).values
y = music_df["genre"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(np.mean(X), np.std(X))
print(np.mean(X_train_scaled), np.std(X_train_scaled))

In [ ]:
# Scaling in a pipeline
steps = [('scaler', StandardScaler()),
         ('knn', KNeighborsClassifier(n_neighbors=6))]
pipeline = Pipeline(steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=21)
knn_scaled = pipeline.fit(X_train, y_train)
y_pred = knn_scaled.predict(X_test)

print(knn_scaled.score(X_test, y_test))

In [ ]:
# Comparing performance using unsacaled data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=21)
knn_unscaled = KNeighborsClassifier(n_neighbors=6).fit(X_train, y_train)
print(knn_unscaled.score(X_test, y_test))

In [ ]:
# Cross-validation and scaling in a pipeline
from sklearn.model_selection import GridSearchCV
steps = [('scaler', StandardScaler()),
         ('knn', KNeighborsClassifier())]
pipeline = Pipeline(steps)
parameters = {"knn__n_neighbors": np.arange(1, 50)}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=21)
cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

## Evaluating multiple models

In [ ]:
# Evaluating calssification models
models = {"Logistic Regression": LogisticRegression(), "KNN": KNeighborsClassifier(), "Decision Tree": DecisionTreeClassifier()}
results = []
for model in models.values():
    kf = KFold(n_splits=6, random_state=42, shuffle=True)
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kf)
    results.append(cv_results)

plt.boxplot(results, labels=models.keys())
plt.show()


# Unsupervised Learning

## K-means clustering
- Finds clusters of samples
- Number of clusters must be specified

### Cluster labels for new samples
- New samples can be assigned to existing clusters
- k-means remembers the mean of each cluster(the "centroids")
- Finds the nearest centroid to each new sample

In [ ]:
# k-means clustering
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3)
model.fit(samples)
labels = model.predict(samples)
print(labels)

In [ ]:
# Visualizing with scatter plot
import matplotlib.pyplot as plt
xs = samples[:,0]
ys = samples[:,2]
plt.scatter(xs, ys, c=labels)
plt.show()

## Evaluating a clustering

In [ ]:
# 1° Aligning labels and species
import pandas as pd
df = pd.DataFrame({'labels':labels, 'species':species})

# 2° Crosstab od labels and species
ct = pd.crosstab(df['labels'], df['species'])

## Measuring clustering quality
- Using only samples and  their cluster labels
- A good clustering has tight clusters
- Samples in each cluster bunched together

### Inertia measures clustering quality
- Measures how spread out the clusters are (lower is better)
- Distance from each sample to centroid of its cluster
- After **fit()**, available as attribute **inertia_**

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3)
model.fit(samples)
print(model.inertia_)

## Transforming features for better clusterings

In [ ]:
# StandardScaler
from sklearn.preprocessing import Standard Scaler
scaler = StandardScaler()
scaler.fit(samples)
StandardScaler(copy=True, with_mean=True, with_std=True)
samples_scaled = scaler.transform(samples)

In [ ]:
# Pipelines combine multiple steps
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
scaler = StandardScaler()
kmeans = KMeans(n_clusters=3)

from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(scaler, kmeans)
pipeline.fit(samples)

labels = pipeline.predict(samples)

## Visualizing hierarchies

In [ ]:
# Hierarchical clustering with SciPy
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
mergings = linkage(samples, method='complete')
dendrogram(mergings=country_names,
         leaf_rotation=90,
         leaf_front_size=6)
plt.show()

In [ ]:
# Extracting cluster labels using fcluster
from scipy.cluster.hierarchy import linkage
mergings = linkage(samples, method='complete')
from scipy.cluster.hierarchy import fcluster
labels = fcluster(mergings, 15, criterion='distance')
print(labels)

In [ ]:
# Aligning cluster labels with country names
import pandas as pd
pairs = pd.DataFrame({'labels':labels, 'countries': country_names})
print(pairs.sort_values('labels'))

## t-SNE for 2-dimensional maps
- t-SNE = "t-distributed stochastic neighbor embedding"
- Maps samples to 2D space (or 3D)
- Map approximately preserves nearness of samples
- Great for inspecting datasets

In [ ]:
# t-SNE in sklearn
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
model = TSNE(learning_rate=100)
transformed = model.fit_transform(samples)
xs = transformed[:,0]
ys = transformed[:,1]
plt.scatter(xs, ys, c=species)
plt.show()

## Principal Component Analysis (PCA)
- Fundamental dimension reduction technique:
1. More efficient storage and computation
2. Remove less-informative "noise" features
3. ...which cause problems for prediction tasks, e.g. classification, regression

### PCA aligns data with axes
- Rotates data samples to be aligned with axes
- Shifts data samples so they have mean 0
- No information is lost

**Obs**: PCA is called "Principal component analysis" because it learns the "principal components" of the data
- "Principal components" = directions of variance
- PCA aligns principal components with the axes

In [ ]:
# Using scikit-learn PCA
from sklearn.decomposition import PCA
model = PCA()
model.fit(samples)
transformed = model.transform(samples)
print(model.components_)

### Intrinsic dimension = number of features needed to approximate the dataset
### Intrinsic dimension = number of PCA features with significant variance
- Essential idea behind dimension reduction
- What is the most compact representation of the samples

In [ ]:
# Can use scipy.sparse.csr_matrix instead NumPy array
from sklearn.decomposition import TruncatedSVD
model = TruncatedSVD(n_components=3)
model.fit(documents)
transformed = model.transform(documents)

## Non-negative matrix factorization (NMF)

In [ ]:
# NMF
from sklearn.decomposition import NMF
model = NMF(n_components=2)
model.fit(samples)
nmf_features = model.transform(samples)

# Visualizing samples
bitmap = sample.reshape((2, 3))
from matplotlib import pyplot as plt
plt.imshow(bitmap, cmap='gray', interpolation='nearest')
plt.show()

# Machine Learning with Tree-Based Models in Python

## Classification-Tees

### General aspects
- Sequence of if-else questions about individual features.
- **Objective**: infer class labels.
- Able to capture non-linear relationships between features and labels.
- Don't require feature scaling (ex:Standardization,..)

### Decisions Regions
- Region in the feature space where all instances are assigned to one class label

### Decision Boundary
- Surface separating different decision regions

### Building blocks of a Decision-Tree
- **Decision-Tree**: data structure consisting of a hierarchy of nodes
- **Node**: question or prediction
- Three kinds of nodes:
1. **Root**: no parent node, question giving rise to two children nodes
2. **Internal node**: one parent node, question giving rise to two children nodes.
3. **Leaf**: one parent node, no children nodes --> prediction

In [ ]:
# Decision tree for classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, 
                                                   stratify=y, random_state=1)
dt = DecisionTreeClassifier(max_depth=2, random_state=1)
# dt = DecisionTreeClassifier(criterion='gini', random_state=1)
dt.fit(X_train,y_train) 
y_pred = dt.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
# Decision tree for regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=3)

dt = DecisionTreeRegressor(max_depth=4,                            
                           min_samples_leaf=0.1,                           
                           random_state=3)
dt.fit(X_train,y_train) 
y_pred = dt.predict(X_test)

mse_dt = MSE(y_test, y_pred)
rmse_dt = mse_dt**(1/2)
print(rmse_dt)

## The Bias-Variance Tradeoff

### Difficulties in Approximating f
- **Overfitting:** 'f'(x) fits the training set noise
- **Underfitting:** 'f' is not flexible enough to approximate f.

## Generalization Error
'f' = bias² + variance + irreducible error

### Bias
- **Bias:** error term that tells you, on average, how much 'f' ≠ f

### Variance
- **Variance:** tells you how much 'f' is inconsistent over different training sets

### Model Complexity
- **Model Complexity:** sets the flexibility of 'f'
![1](1.webp)

In [ ]:
# K-Fold CV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score

SEED = 123
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=SEED)
dt = DecisionTreeRegressor(max_depth=4, 
                           min_samples_leaf=0.14, 
                           random_state=SEED)

MSE_CV = - cross_val_score(dt, X_train, y_train, cv= 10,
                           scoring='neg_mean_squared_error',
                           n_jobs = -1)
dt.fit(X_train, y_train)
y_predict_train = dt.predict(X_train)
y_predict_test = dt.predict(X_test)

# CV MSE
print('CV MSE: {:.2f}'.format(MSE_CV.mean()))
# Training set MSE
print('Train MSE: {:.2f}'.format(MSE(y_train, y_predict_train)))
# Test set MSE
print('Test MSE: {:.2f}'.format(MSE(y_test, y_predict_test)))

## Ensemble Learnig
- Train dierent models on the same dataset.
- Let each model make its predictions.
- Meta-model: aggregates predictions of individual models.
- Final prediction: more robust and less prone to errors.
- Best results: models are skillful in dierent ways.

In [ ]:
# Voting Classfier in sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import VotingClassifier

SEED = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3,
                                                    random_state= SEED)
lr = LogisticRegression(random_state=SEED)
knn = KNN()
dt = DecisionTreeClassifier(random_state=SEED)
classifiers = [('Logistic Regression', lr),
               ('K Nearest Neighbours', knn),
               ('Classification Tree', dt)]

# Iterate over the defined list of tuples containing the classifiers
for clf_name, clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('{:s} : {:.3f}'.format(clf_name, accuracy_score(y_test, y_pred)))

# Instantiate a VotingClassifier 'vc'
vc = VotingClassifier(estimators=classifiers) 
vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)
print('Voting Classifier: {.3f}'.format(accuracy_score(y_test, y_pred)))

## Bagging
- Bagging: Bootstrap Aggregation . 
- Uses a technique known as the bootstrap. 
- Reduces variance of individual models in the ensemble.

In [ ]:
# Bagging Classifier in sklearn
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

SEED = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
stratify=y, random_state=SEED)

dt = DecisionTreeClassifier(max_depth=4, min_samples_leaf=0.16, random_state=SEED)
bc = BaggingClassifier(base_estimator=dt, n_estimators=300, n_jobs=-1)

bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

## Random Forest Regressor

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=SEED)
f = RandomForestRegressor(n_estimators=400,
                          min_samples_leaf=0.12,
                          random_state=SEED)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
rmse_test = MSE(y_test, y_pred)**(1/2)
print('Test set RMSE of rf: {:.2f}'.format(rmse_test)

In [ ]:
# Feature Importance in sklearn
import pandas as pd
import matplotlib.pyplot as plt
importances_rf = pd.Series(rf.feature_importances_, index = X.columns)
sorted_importances_rf = importances_rf.sort_values()
sorted_importances_rf.plot(kind='barh', color='lightgreen')
plt.show()

## AdaBoost (Adaptative Boosting)
- Train an ensemble of predictors sequentially.
- Each predictor tries to correct its predecessor.
- Each predictor pays more attention to the instances wrongly predicted by its predecessor.
- Achieved by changing the weights of training instances.
- Each predictor is assigned a coefficient _a_.
- _a_ depends on the predictor training error.

In [ ]:
# AdaBoost Classification in sklearn
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    stratify=y,
                                                    random_state=SEED)
dt = DecisionTreeClassifier(max_depth=1, random_state=SEED)
adb_clf = AdaBoostClassifier(base_estimator=dt, n_estimators=100)

adb_clf.fit(X_train, y_train)
y_pred_proba = adb_clf.predict_proba(X_test)[:,1]
adb_clf_roc_auc_score = roc_auc_score(y_test, y_pred_proba)

print('ROC AUC score: {:.2f}'.format(adb_clf_roc_auc_score))

## Gradient Boosting (GB)
- Sequential correction of predecessor's erros.
- Does not tweak the weights of training intances.
- Fit each predictor is trained using its predecessor's residual errors as labels.
- Gradient Boosted Trees: a CART is used as a base learner.

In [ ]:
# Gradient Boosting in sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=SEED)

gbt = GradientBoostingRegressor(n_estimators=300, max_depth=1, 
                                random_state=SEED)
gbt.fit(X_train, y_train)
y_pred = gbt.predict(X_test)
rmse_test = MSE(y_test, y_pred)**(1/2)
print('Test set RMSE: {:.2f}'.format(rmse_test)

## Stochastic Gradient Boosting (SGB)
- Each tree is trained on a random subset of rows of the training data.
- The sampled instances (40% - 80% of the training set) are sampled without replacement.
- Features are sampled (without replacement) when choosing split points.
- Result: further ensemble diversity
- Effect: adding further variance to the ensemble of trees

In [ ]:
# Stochastic Gradient Boosting in sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

SEED = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=SEED)
sgbt = GradientBoostingRegressor(max_depth=1, 
                                 subsample=0.8,
                                 max_features=0.2,
                                 n_estimators=300,
                                 random_state=SEED)
gbt.fit(X_train, y_train)
y_pred = sgbt.predict(X_test)

rmse_test = MSE(y_test, y_pred)**(1/2)
print('Test set RMSE: {:.2f}'.format(rmse_test))

## Hyperparameters
- **Parameters:** learnd from data
- **Hyperparameters:** not learned from data, set prior to training

In [ ]:
# Inspecting the hyperparameters of a CART in sklearn
from sklearn.tree import DecisionTreeClassifier

SEED = 1
dt = DecisionTreeClassifier(random_state=SEED)
print(dt.get_params())

In [ ]:
from sklearn.model_selection import GridSearchCV
params_dt = {
    'max_depth': [3, 4, 5, 6], 
    'min_samples_leaf': [0.04, 0.06, 0.08], 
    'max_features': [0.2, 0.4, 0.6, 0.8]
}
grid_dt = GridSearchCV(estimator=dt,
                       param_grid=params_dt, 
                       scoring='accuracy', 
                       cv=10, 
                       n_jobs=-1)
grid_dt.fit(X_train, y_train)

# Extracting the best hyperparameters
best_hyperparams = grid_dt.best_params_
print('Best hyerparameters:\n', best_hyperparams)
best_CV_score = grid_dt.best_score_
print('Best CV accuracy'.format(best_CV_score))Best CV accuracy: 0.938

# Extracting the best estimator
best_model = grid_dt.best_estimator_
test_acc = best_model.score(X_test,y_test)
print("Test set accuracy of best model: {:.3f}".format(test_acc))

In [ ]:
# Inspecting Random Forest Hyperparameters in sklean
from sklearn.ensemble import RandomForestRegressor

SEED = 1
rf = RandomForestRegressor(random_state= SEED)
print(rf.get_params())

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
params_rf = {
    'n_estimators': [300, 400, 500], 
    'max_depth': [4, 6, 8], 
    'min_samples_leaf': [0.1, 0.2], 
    'max_features': ['log2', 'sqrt']
}
grid_rf = GridSearchCV(estimator=rf,
                       param_grid=params_rf,
                       cv=3, 
                       scoring='neg_mean_squared_error', 
                       verbose=1, 
                       n_jobs=-1)
grid_rf.fit(X_train, y_train)
# Extracting the best hyperparameters
best_hyperparams = grid_rf.best_params_
print('Best hyperparameters:\n', best_hyperparams)

# Evaluating the model performance
best_model = grid_rf.best_estimator_
y_pred = best_model.predict(X_test)
rmse_test = MSE(y_test, y_pred)**(1/2)
print('Test set RMSE of rf: {:.2f}'.format(rmse_test))